In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
import VertRegridFlexLL as Vrg
import PlotUtil as Pu
import validation_data as Val

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)


In [ ]:

yearA = "2000"
monA  = "2000-10"
ERA5path ='/glade/derecho/scratch/juliob/ERA5/ne30pg3/'

ERA5files = ERA5path + "ERA5_x_ne30pg3_L93_Test01_WO." + monA + "*.nc"
# a file
#    ERA5_x_ne30pg3_L93_Test01_WO.2001-01-11-82800.nc

ERA = xr.open_mfdataset( ERA5files, concat_dim='time' ,combine='nested' )# ,data_vars='different', coords='different' )

In [ ]:
ERA

In [ ]:
#####################
user_C,user_X='juliob','juliob'

subd='regridded'
htap='cam.h0'

#subd='regridded'


exp_C = 'test3.02.2'  # Overall control for tag=cam6_3_139
exp_X = 'fmtndg_S2_x04'
exp_C = 'fmtndg_N_x03' #_rd1p0_cd0p1'

path_C = '/glade/derecho/scratch/juliob/archive/'+exp_C+'/atm/'+ subd +'/'+exp_C+'.'+ htap + '.*.nc' 
path_X = '/glade/derecho/scratch/juliob/archive/'+exp_X+'/atm/'+ subd +'/'+exp_X+'.'+ htap + '.*.nc' 



In [ ]:
###

Dc = xr.open_mfdataset( path_C ,data_vars='different', coords='different' )
Dx = xr.open_mfdataset( path_X ,data_vars='different', coords='different' )


In [ ]:
zlev=-7.0*np.log( Dx.lev.values/1_000. )


In [ ]:
Dx

In [ ]:
sqP = np.sqrt(Dx.UPWP_CLUBB**2+Dx.VPWP_CLUBB**2 )

Dx['MOMFLXMAG_CLUBB'] = sqP

Dx['STEND_CLUBB'] = Dx['STEND_CLUBB'] * 86400./1002.

In [ ]:

CLUBBparms = ['MOMFLXMAG_CLUBB','WP2_CLUBB','STEND_CLUBB'] #'THLP2_CLUBB']


In [ ]:
######################
# Pretty latlon plot
######################

#MapProj = ccrs.Robinson(central_longitude=180.)
MapProj = ccrs.PlateCarree(central_longitude=180.)
#MapProj = ccrs.Orthographic(central_longitude=180.,central_latitude=-90.)
DataProj = ccrs.PlateCarree()

"""
Add axes method 
ax = fig.add_axes([xmin , ymin , dx , dy , ])
"""

fig = plt.figure(figsize=(20, 12))
levs=[29,26,20,10]
   
monAs=[ 'jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec' ]    
mon=9 -1
cmap='gist_ncar'
npo=0

season=monAs[mon%12]

kl0,kl1=78,92
zA = f" ( {Dx.lev[kl0]:.0f}-{Dx.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"

CoastColors=['white','white','white','black']
clevs = [21,21,np.linspace(0,20,21),21]

for CLUBBparm in CLUBBparms:
    npo=npo+1
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 

    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color=CoastColors[npo-1],linewidth=2)


    AAxy = np.average(Dx[CLUBBparm][mon,kl0:kl1,:,:],axis=0)

    co1=ax1.contourf(Dx.lon,Dx.lat, AAxy ,transform=DataProj,levels=clevs[npo-1],cmap=cmap)

    cbar = plt.colorbar(co1, shrink=.6)
    #ax1.set_title( CLUBBparm + zA, fontsize=16)
    ax1.set_title( f"{season.upper()} {CLUBBparm} {zA}" , fontsize=16)

npo=npo+1
Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color=CoastColors[npo-1])

kl0,kl1=11,22
#kl0,kl1= 4,10 #11,22
#zA = f" ( {Dx.lev[kl0]:.2f} hPa)"
zA = f" ( {Dx.lev[kl0]:.0f}-{Dx.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"

AAxy = 86400.*np.average(Dx.Nudge_U[mon,kl0:kl1,:,:],axis=0)
ndglev=5*np.linspace( -2,2,num=21) 


co2=ax1.contourf(Dx.lon,Dx.lat, AAxy ,transform=DataProj,levels=.2*ndglev,cmap='bwr')

cbar = plt.colorbar(co2, shrink=.6)


ax1.set_title( f"{season.upper()} Nudge_U {zA}" , fontsize=16)



In [ ]:
######################
# Pretty latlon plot
######################

#MapProj = ccrs.Robinson(central_longitude=180.)
#MapProj = ccrs.PlateCarree(central_longitude=180.)
DataProj = ccrs.PlateCarree()

"""
Add axes method 
ax = fig.add_axes([xmin , ymin , dx , dy , ])
"""

fig = plt.figure(figsize=(20, 12))
levs=[29,26,20,10]
   
monAs=[ 'jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec' ]    
mon=7 -1
kl0,kl1=11,22
cmap='gist_ncar'
npo=0

season=monAs[mon%12]


CoastColors=['black'  , 'black'  , 'black'  , 'black'  , 'black'  ,]

npo=npo+1
Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color=CoastColors[npo-1])

zA = f" ( {Dx.lev[kl0]:.0f}-{Dx.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"

AAxy = 86400.*np.average(Dc.Nudge_U[mon,kl0:kl1,:,:],axis=0)
ndglev=5*np.linspace( -2,2,num=21) 


co2=ax1.contourf(Dc.lon,Dc.lat, AAxy ,transform=DataProj,levels=0.2*ndglev,cmap='bwr')

cbar = plt.colorbar(co2, shrink=.6)


ax1.set_title( f"{exp_C} {season.upper()} Nudge_U {zA}" , fontsize=16)

npo=npo+1
Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 
ax1 = fig.add_axes( Axes , projection=MapProj)
ax1.set_global()
ax1.coastlines(resolution='110m',color=CoastColors[npo-1])

zA = f" ( {Dx.lev[kl0]:.0f}-{Dx.lev[kl1]:.0f} hPa, {zlev[kl1]:.0f}-{zlev[kl0]:.0f} km)"

AAxy = 86400.*np.average(Dx.Nudge_U[mon,kl0:kl1,:,:],axis=0)  #-AAxy
ndglev=5*np.linspace( -2,2,num=21) 


co2=ax1.contourf(Dx.lon,Dx.lat, AAxy ,transform=DataProj,levels=0.2*ndglev,cmap='bwr')

cbar = plt.colorbar(co2, shrink=.6)


ax1.set_title( f"{exp_X} {season.upper()} Nudge_U {zA}" , fontsize=16)



In [ ]:
#plt.plot(Dx.wp3[0,90,:])
plt.contourf(Dx.lon,Dx.lat,np.average(Dx.UPWP_CLUBB[9,80:,:],axis=0),levels=21)
plt.colorbar()

In [ ]:
#plt.plot(Dx.wp3[0,90,:])
plt.tricontourf(Dx.lon,Dx.lat,np.average(Dx.UPWP_CLUBB[9,80:,:],axis=0),levels=21)
plt.colorbar()

In [ ]:
plt.plot( Dx.wpthlp[900:910,85,:].values.flatten())

In [ ]:
plt.scatter( Dx.wp3[890:900,85,:].values.flatten() ,  Dx.Nudge_U[900:910,5,:].values.flatten()  )